In [1]:
# 사전 훈련 모델 : VGG16 모델
# 8-19. VGG16 합성곱 기반 층 만들기
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

conv_base.summary()

NameError: name 'keras' is not defined

*이탤릭체 텍스트*
 사전 훈련된 모델을 사용한 특성 추출
⚫ VGG16 함수에 3개의 매개변수를 전달
 모델을 초기화할 imagenet 가중치 - weights

 밀집 연결 분류기를 포함 (기본값) - include_top = true

 강아지와 고양이를 구분하는 밀집 연결 층을 훈련시키기 위해 imagenet 가중치는 불필요
 include_top = false

네트워크에 주입할 이미지 텐서의 크기
이 값을 지정하지 않으면 어떤 크기의 입력도 처리할 수 있음
input_shape

In [2]:
# 8-20. VGG16 특성과 해당 레이블 추출하기
import numpy as np

def get_features_and_labels(dataset):
  all_features = []
  all_labels = []
  for images, labels in dataset:
    preprocessed_images = keras.applications.vgg16.preprocess_input(images)
    features = conv_base.predict(preprocessed_images)
    all_features.append(features)   # 5,5,512 목록 끝에 요소를 추가, 각 요소는 세트의 각 이미지 배치에 해당하는 기능 배열
    all_labels.append(labels), np.concatenate(all_labels)     # (samples, 5, 5, 512) 단일 연결된 배열 - 표준화되고 편리한 표현 제공

train_features,train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

train_features.shape

NameError: name 'train_dataset' is not defined

In [5]:
# 8-21. 밀집 연결 분류기 정의하고 훈련하기
inputs = keras.Input(shape=(5,5,512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction.keras",
        save_best_only=True,
        monitor="val_loss")
]

# 사전 훈련된 모델을 사용한 특성 추출
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data(val_features, val_labels),
    callbacks=callbacks)

# 8-22. 결과를 그래프로 나타내기
import matplotlib.pyplot as plt

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

SyntaxError: positional argument follows keyword argument (<ipython-input-5-4b428e2ca5a7>, line 24)

In [6]:
# 8-23 VGG16 합성곱 기반 층을 만들고 동결하기
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)

# 8-24 동결하기 전과 후에 훈련 가능한 가중치 리스트 출력
conv_base.trainable = True
print("합성곱 기반 층을 동결한 후의 훈련 가능한 가중치 개수:",
      len(conv_base.trainable_weights))

conv_base.trainable = False
print("합성곱 기반 층을 동결한 후의 훈련 가능한 가중치 개수:",
      len(conv_base.trainable_weights))

# 8-25 데이터 증식 단계와 밀집 분류기를 합성곱 기반 층에 추가하기
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

# 8-26 테스트 세트에서 모델 평가하기
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"테스트 정확도: {test_acc:.3f}")

NameError: name 'keras' is not defined